In [1]:
!pip install tabula
!pip install --upgrade PyPDF2
!pip install PyDrive
!pip install fronted 
!pip install pyserial 
!pip install tools 
!pip install pdfminer
!pip install textblob
!pip install PyMuPDF
!pip install pdfminer.six
!pip install fpdf
!!pip install frontend
!pip install re
!pip install fitz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tabula: filename=tabula-1.0.5-py3-none-any.whl size=10585 sha256=8ad7d8fdf1e727d1fe13c94ec96162b6e5020ba5c260e4fd5b6ab5c49be0d4bf
  Stored in directory: /root/.cache/pip/wheels/3c/5d/6a/bd13fe26e73e2170dfea1f6ccb69a69e824735e04b73192b39
Successfully built tabula
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement fronted (from versions: none)
ERROR: No matching distribution found for fronted
Looking in indexes: https://pypi.org/simple, https://us-

In [2]:
import os

# Create the 'static/' directory if it doesn't exist
if not os.path.exists('static/'):
    os.makedirs('static/')

In [3]:
# Step 1: Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [9]:
import PyPDF2
from PyPDF2 import PdfReader
import textblob
import operator
import fitz
import re
import operator
import builtins
class PDFDissector():
    @staticmethod
    def input_file():
        pdf_filename = " "

        try:
            PyPDF2.PdfReader(pdf_filename)
        except FileNotFoundError:
            print("File not found! Try again")
            PDFDissector.input_file()

        return pdf_filename


def select_option():
    print("1.   Perform sentiment analysis on the entire document")
    print("2.   Perform sentiment analysis on a specific page")
    print("3.   Perform heading-wise sentiment analysis")
    print("4.   Perform per sentence sentiment analysis")  

    selected_option = input("What would you like to do? ")

    return selected_option



def perform_document_analysis(pdf_filename):
    analysis_result = ""
    document_text = ""
    document_pages = PyPDF2.PdfReader(pdf_filename).pages

    for page in range(len(document_pages)):
        document_text += document_pages[page].extract_text()
    
    document_polarity = textblob.TextBlob(document_text).sentiment.polarity

    if -1 <= document_polarity <= -0.5:
        analysis_result = "The document is  negative"
    elif -0.5 < document_polarity <= -0.01:
        analysis_result = "The document is slightly negative"
    elif document_polarity == 0:
        analysis_result = "The document is neutral"
    elif 0.01 < document_polarity <= 0.5:
        analysis_result = "The document slightly positive"  
    elif 0.5 < document_polarity <= 1:
        analysis_result = "The document is positive"

    print("Document Sentiment Analysis Result:")
    print(analysis_result)
    
def perform_heading_wise_sentiment_analysis(pdf_filename):
    pdf_file_object = PyPDF2.PdfReader(pdf_filename)
    heading_pattern = r'(?<=\n)[IV]+\.\s*.*?(?=\n)'  # Regex pattern to detect headings
    check_reference = r'References|REFERENCES'  # Regex pattern to detect word "References" in text
    text = " "
    reference = re.findall(check_reference, text, re.M)  # Find the word References

    with open(pdf_filename, "rb") as f:
        pdf = PyPDF2.PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

    headers = re.findall(heading_pattern, text, re.M)  # Find all the headings present in the text

    if reference:
        reference_index = text.find(reference[0])
    else:
        reference_index = len(text)

    for heading in headers:
        start_index = text.find(heading)
        end_index = reference_index if start_index < reference_index else len(text)

        heading_text = text[start_index:end_index]

        heading_polarity = textblob.TextBlob(heading_text).sentiment.polarity

        if -1 <= heading_polarity <= -0.5:
            analysis_result = "The heading is negative"
        elif -0.5 < heading_polarity <= -0.01:
            analysis_result = "The heading is slightly negative"
        elif -0.01 < heading_polarity <= 0.01:
            analysis_result = "The heading is neutral"
        elif 0.01 < heading_polarity <= 0.5:
            analysis_result = "The heading is slightly positive"
        elif 0.5 < heading_polarity > 1:
            analysis_result = "The heading is positive"

        print("Heading: ", heading)
        #print("Heading Text: ", heading_text)
        print("Heading Sentiment Analysis Result:", analysis_result)
        print()


def perform_page_analysis(pdf_filename):
    page_number = int(input("Enter the page number for sentiment analysis: ")) - 1
    pdf_file_object = PyPDF2.PdfReader(pdf_filename)
    page_object = pdf_file_object.pages[page_number]
    page_text = page_object.extract_text()

    page_polarity = textblob.TextBlob(page_text).sentiment.polarity

    if -1 <= page_polarity <= -0.5:
        analysis_result = "The page is negative"
    elif -0.5 < page_polarity <= -0.01:
        analysis_result = "The page is slightly negative"
    elif page_polarity == 0:
        analysis_result = "The page is neutral"
    elif 0.01 < page_polarity <= 0.5:
        analysis_result = "The page is slightly positive"
    elif 0.5 < page_polarity > 1:
        analysis_result = "The page is positive"

    print("Page Sentiment Analysis Result: ",analysis_result)



def perform_sentence_analysis(pdf_filename):
    pdf_file_object = PyPDF2.PdfReader(pdf_filename)
    text = ""

    for page in pdf_file_object.pages:
        text += page.extract_text()

    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)

    for sentence in sentences:
        sentence_polarity = textblob.TextBlob(sentence).sentiment.polarity

        if -1 <= sentence_polarity <= -0.5:
            analysis_result = "The sentence is negative"
        elif -0.5 < sentence_polarity <= -0.01:
            analysis_result = "The sentence is slightly negative"
        elif -0.01 < sentence_polarity <= 0.01:
            analysis_result = "The sentence is neutral"
        elif 0.01 < sentence_polarity <= 0.5:
            analysis_result = "The sentence is slightly positive"
        elif sentence_polarity > 0.5:
            analysis_result = "The sentence is positive"

        print("Sentence: ", sentence)
        print("Sentence Sentiment Analysis Result: ", analysis_result)
        print()


def main():
    pdf_filename = input("Enter the name of the PDF file: ")

    while True:
        option = select_option()

        if option == "1":
            perform_document_analysis(pdf_filename)
        elif option == "2":
            perform_page_analysis(pdf_filename)
        elif option == "3":
            perform_heading_wise_sentiment_analysis(pdf_filename)       
        elif option == "4":
            perform_sentence_analysis(pdf_filename)  # Added option
        else:
            print("Invalid option. Please try again.")

        choice = input("Do you want to continue? (y/n) ")
        if choice.lower() != "y":
            break


if __name__ == "__main__":
    main()



Enter the name of the PDF file: /content/gdrive/MyDrive/A/RP.pdf
1.   Perform sentiment analysis on the entire document
2.   Perform sentiment analysis on a specific page
3.   Perform heading-wise sentiment analysis
4.   Perform per sentence sentiment analysis
What would you like to do? 4
Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis Result:  The sentence is neutral

Sentence Sentiment Analysis 